## Introduction

### What is Function Calling

Function calling enables Large Language Models (LLMs) to interact with external systems, execute tasks, and integrate with APIs. The LLM determines when to invoke a function based on user prompts and returns structured data for app use. Developers then implement the function logic within the app.

In this workshop, the function logic is used to execute the LLM dynamically generated SQL queries against the SQLite database.

### Enabling Function Calling

If you’re familiar with Azure OpenAI Function Calling, it requires defining a function schema for the LLM. Azure AI Agent Service supports this approach and also offers a more flexible option.

With the Azure AI Agent Service and its Python SDK, you can define the function schema directly within the Python function’s docstring. This approach keeps the definition and implementation together, simplifying maintenance and enhancing readability.

For example, in the sales_data.py file, the async_fetch_sales_data_using_sqlite_query function uses a docstring to specify its signature, inputs, and outputs. The SDK parses this docstring to generate the callable function for the LLM:

```python
async def async_fetch_sales_data_using_sqlite_query(self: "SalesData", sqlite_query: str) -> str:
    """
    This function is used to answer user questions about Contoso sales data by executing SQLite queries against the database.

    :param sqlite_query: The input should be a well-formed SQLite query to extract information based on the user's question. The query result will be returned as a JSON object.
    :return: Return data in JSON serializable format.
    :rtype: str
    """
```

### Dynamic SQL Generation

When the app starts, it incorporates the database schema and key data into the instructions for the Azure AI Agent Service. Using this input, the LLM generates SQLite-compatible SQL queries to respond to user requests expressed in natural language.

### Lab Exercise

In this lab, you'll enable the function logic to execute dynamic SQL queries against the SQLite database. The function will be called by the LLM to answer user questions about Contoso sales data.

1. Open the main.py.

2. Uncomment the following lines by removing the "# " characters

```python
    # INSTRUCTIONS_FILE = "instructions/instructions_function_calling.txt"
    # toolset.add(functions)
```

### Run the Agent App

Start a Conversation with the Agent

Start asking questions about Contoso sales data. For example:

1. Help

Here is an example of the LLM response to the help query:

```
I’m here to help with your sales data inquiries at Contoso. Could you please provide more details about what you need assistance with? Here are some example queries you might consider:

- "What were the sales by region?"
- "What was last quarter's revenue?"
- "Which products sell best in Europe?"
- "Total shipping costs by region?"

Feel free to ask any specific questions related to Contoso sales data!
```

2. What are the sales by region?

Here is an example of the LLM response to the sales by region query:

```markdown
| Region         | Total Revenue  |
|----------------|----------------|
| AFRICA         | $5,227,467     |
| ASIA-PACIFIC   | $5,363,718     |
| CHINA          | $10,540,412    |
| EUROPE         | $9,990,708     |
| LATIN AMERICA  | $5,386,552     |
| MIDDLE EAST    | $5,312,519     |
| NORTH AMERICA  | $15,986,462    |
```